# CE9010 Introduction To Data Analysis

## Group 3

Student Name  |  Matric No  
:-:|:-:
Say Yueyang, Symus|U1922016K   
He Zeqing|U1722721G
Kwek Yan Qing|U1740743J   

---

# Background

Haze is one major form of air pollution that Singaporeans face annually. The presence of haze is attributed to the forest fires in Sumatra, Indonesia. Due to the increase in demand of commercial crops, Indonesian farmers have resorted to shifting agriculture, which includes the large-scale slash-and-burn of forest land to produce fertile planting grounds. The resulting air pollution is then spread across the region by the climatic phenomenon El Nino, enveloping Singapore in a blanket of haze and affecting the overall health of Singaporeans. In 2020, Indonesia fires torched approximately 207,000 hectares of forests from January to September. While the area is smaller compared to previous years, the burning resulted in a US$5.2 billion cost towards the Indonesian economy, and the occurrence of toxic smog over the city.

**References:**
- [Haze Pollution](https://eresources.nlb.gov.sg/infopedia/articles/SIP_2013-08-30_185150.html#:~:text=Forest%20fires%20in%20Sumatra%2C%20Indonesia,of%20the%20haze%20in%20Singapore.&text=Strong%20winds%20during%20the%20southwest,such%20fires%20throughout%20Southeast%20Asia.)
- [Commentary: Little smoke this haze season – but fires rage on in Indonesia](https://www.channelnewsasia.com/news/commentary/indonesia-forest-fire-peat-haze-palm-oil-jokowi-omnibus-bill-13533700)

# Objective

The objective of our study is to predict the possible intensity of future hotspots in South East Asia, including Indonesia.
Hopefully, this study will be able to support further research in estimating the possibility and severity of the occurrences of haze in Singapore.

Our study will be conducted with the relevant data on forest fires in South East Asia. Our dataset is obtained from the National Aeronautics and Space Administration (NASA)'s Fire Information for Resource Management System (FIRMS). It contains both geographical and technical data extracted from the Visible Infrared Imaging Radiometer Suite (VIIRS) sensor aboard their NOAA-20 weather satellite.

The table below describes each data available in our dataset:

| Data | Description |
| -: | :- |
| latitude | Indicates the latitude of fire pixel. |
| longitude	| Indicates the longitude of fire pixel. |
| bright_ti4 | Indicates the VIIRS I-4 Channel brightness temperature of the fire pixel. |
| scan | Indicates the  Along Scan pixel size. |
| track	| Indicates the Along Track pixel size.  |
| acq_date | Indicates the date of the acquired data. |
| acq_time | Indicates the time of the acquired data. |
| satellite | Indicates if the scan was done by the satellite (boolean values). |
| confidence | Indicates the confidence level of the data collected. |
| version | Indicates the version and source of data processing. |
| bright_ti5 | Indicates the VIIRS I-5 Channel brightness temperature of the fire pixel. |
| frp | Indicates the Fire Radiative Power (Detected thermal strength of the fire). |
| daynight | Indicates whether if it's daytime fire or nighttime fire. |

**References:**
- [Fire Information for Resource Management System](https://firms2.modaps.eosdis.nasa.gov/)
- [Visible Infrared Imaging Radiometer Suite](https://en.wikipedia.org/wiki/Visible_Infrared_Imaging_Radiometer_Suite)
- [Attribute Fields](https://earthdata.nasa.gov/earth-observation-data/near-real-time/firms/v1-vnp14imgt#ed-viirs-375m-attributes)

---

# Table of Contents

1. [Setup](#1-|-Setup)
2. [Dataset Summaries](#2-|-Dataset-Summaries)
3. [Exploratory Data Analysis](#3-|-Exploratory-Data-Analysis)
4. [Data Pre-Preprocessing](#4-|-Data-Pre-Processing)
5. [Data Analysis](#5-|-Data-Analysis)
6. [Results Analysis](#6-|-Results-Analysis)

# 1 | Setup 

## 1.1 | Setup on Anaconda Prompt / Local Computer

1. Ensure that the environment.yml file accompanying this notebook is located in the same directory as the notebook. <br>
2. Open Anaconda Prompt, and in Anaconda Prompt, navigate to the directory where the notebook was downloaded. <br>
3. In Anaconda Prompt, enter the following line of code:
> conda env create -f environment.yml

4. In Anaconda Prompt, enter the following line of code:
> conda info --envs

If installation of the environment is successful, you will see the environment 'CE9010_2021_Group3' listed in the list of environments.

5. Activate the notebook: 
> conda activate CE9010_2021_Group3

6. Run Jupyter Notebook 
> jupyter notebook

## 1.2 | Setup on Google Colab
Run the following cells to: 

1.   !pip install all the required packages, and
2.   Setup connection to Google Drive as your directory (Optional, for accessing files)

In [ ]:
# For use on Google Colab
import sys
!pip install numpy
!pip install seaborn
!pip install plotly
!pip install pandas
!pip install geopandas
!pip install rtree
!pip install pygeos
!pip install imageio
!pip3 install rtree

In [ ]:
# Set up Google Drive to be the working directory
from google.colab import drive # import drive from Google Colab
 
ROOT = "/content/drive/"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
 
drive.mount(ROOT)           # we mount the Google Drive at /content/drive

In [ ]:
%pwd #check that it is correctly mounted
%ls #list all the directories available

In [ ]:
#Set github repo path
# import join used to join ROOT path and MY_GOOGLE_DRIVE_PATH
from os.path import join 
# path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = "/content/drive/MyDrive/rootCE9010/repo" 
 
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)
 
# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)

%cd {PROJECT_PATH}

In [ ]:
%pwd
%ls

## 1.3 | Import modules
Run this to import all the modules required, regardless of what environment you are running in. Make sure that the imports have no errors before proceeding.

In [ ]:
# Import required modules
import os
try:
    import pandas as pd
    import plotly.express as px
    from matplotlib import pyplot as plt
    from IPython.display import IFrame,Image
    import seaborn as sns
    import numpy as np
    import imageio
    import geopandas as gpd
    import rtree
    import pygeos
    print ("All modules imported successfully.")
except ImportError:
    print ("One or more modules not imported!")
    print ("Please check that all dependencies are installed.")

# Directory to store local content for loading of interactive images
if not os.path.exists("content"):
    os.mkdir("content")
# Clean up past images
else:
    for f in os.listdir('./content'):
        os.remove(os.path.join('content', f))

# 2 | Dataset Summaries 

In [ ]:
# Data Acquisition
# Import data from the Active Fire Dataset, VIIRS 375m / NOAA-20
data = pd.read_csv("https://firms2.modaps.eosdis.nasa.gov/data/active_fire/noaa-20-viirs-c2/csv/J1_VIIRS_C2_SouthEast_Asia_7d.csv",sep=',')
print (data.shape) # dimensions
data[:5]

In [ ]:
# Check dataset
data.describe()

In [ ]:
# Check datatypes
data.dtypes

In [ ]:
# Check for null values (values contain no info and can be removed)
data.isnull().sum()

## 2.1 | Pre-visualization cleanup
We can see that there are no NULL values, indicating clean data. However, the datatypes of certain columns need to be corrected for appropriate data analysis.

In [ ]:
# Concatenate acquisition date and time into a single column
data['period']=data['acq_date']+' '+data['acq_time'].astype(str) # this leaves a df with acq_date and acq_time still there
# data.drop(columns=['acq_date','acq_time'], inplace=True)
data['period']=pd.to_datetime(data['period'], format='%Y-%m-%d %H%M')
data['acq_date']=pd.to_datetime(data['period'].dt.date, format='%Y-%m-%d')
data['acq_time']=data['period'].dt.time
data.sort_values(by=['period'], inplace=True) # observe that without this code, time does not flow correctly in the animation
# data.set_index('period', inplace=True) # sets the index of the dataframe to be the period
data[:10]

# 3 | Exploratory Data Analysis/Visualization
In this section, we shall be doing some preliminary visualization of our dataset.

In [ ]:
# Write animation to file
fig1 = px.scatter_geo(data, 
                    lat='latitude', 
                    lon='longitude', 
                    scope='asia',
                    center={'lat':2.2180,'lon':115.6628}, # centered to SEA
                    color='confidence',
                    animation_frame=data['period'].astype(str)) 
fig1.write_html('content/animation.html')
# TODO: Fix animation to have constant legend 

# Display animation
IFrame(src='content/animation.html', width=1080, height=720)

In [ ]:
# FRP/confidence against time
sns.relplot(x="period", y="frp", hue="confidence", col="daynight", data=data, height=8)

We note that there seems to be some data where the gaps between data is small. Hence, this necessitates the merging of time data into hourly frames to better analyze patterns. 

In [ ]:
# Processing data further to clean visualization

# Generate new DFs with times rounded down to the nearest hour
date_sorted = data
date_sorted['period'] = date_sorted['period'].dt.floor('H')
date_sorted_gb = date_sorted.groupby('acq_date') # returns a groupby object which can be called with below code

# [date_sorted.get_group(x) for x in date_sorted_gb.groups] # this displays all the dataframes

date_sorted.head(10)

In [ ]:
# Plot individual plots for every date
import numpy as np
pd.options.mode.chained_assignment = None  # ignore warnings
# list_date = date_sorted['acq_date'].unique()

# Formatting of plot
fig = plt.figure(figsize=(12,8))
plt.xlabel('Time of Day (24H Format)')
plt.ylabel('FRP')
plt.xlim(0,2400)
plt.xticks(np.linspace(0,2300,num=24), rotation=45)

for x in date_sorted_gb.groups:
    plt.title(str(x.date()))

    # Data of plot
    current = date_sorted_gb.get_group(x) # iterate through groups
    current['acq_time'] = current['period'].dt.time # extract time
    current['acq_time'] = current['acq_time'].apply(str) # convert to type string
    current['acq_time'] = current['acq_time'].str.replace(':','').astype(int)/100 # convert to 24h format
    plt.scatter(current['acq_time'],current['frp'])
    plt.savefig('content/'+str(x.date())+'.png')

In [ ]:
filenames = date_sorted['acq_date'].dt.date.unique().astype(str)
filenames = [(value+'.png') for value in filenames]

images = []
for filename in filenames:
    images.append(imageio.imread('content/'+filename))
imageio.mimwrite('content/dailyfrp.gif', images, format='gif', duration=1)

for item in images:
    display(Image(data=item))

Looking at the above visualizations, we can see that: 
- there tends to be a concentration of fire data within the same region
- most fires are detected in the day
- there seems to be a pattern in when the fires are detected within the same 7 day period

We also notice a few points where the data is seemingly in the middle of the ocean.

Consequently, this necessitates cleaning of data to remove unnecessary information. Feature selection is necessary. 

# 4 | Data Pre-Processing

## 4.1 | Cleaning Dataset

### 4.1.1 | Reformatting Features
Certain columns need to be reformatted into their appropriate data type for analysis.

In [ ]:
# Convert 'object' columns into appropriate dtype
data['confidence'].astype('category')
data['daynight'].astype('category')

In [ ]:
# Convert time into integers
data['acq_time'] = data['acq_time'].apply(str) # convert to type string
data['acq_time'] = data['acq_time'].str.replace(':','').astype(int)
data.head()

Certain columns of categorical data have multiple classifications in one column. To perform regression, categorical data need to be separated out into binary columns of dummy variables. To do so, we shall be using a tool known as OneHotEncoding (OHE).

Source: [Dummy Variables](https://stats.idre.ucla.edu/spss/faq/coding-systems-for-categorical-variables-in-regression-analysis-2/)

In [ ]:
# Resetting index for OHE preparation
data.set_index('period', inplace=True)
data.reset_index(inplace=True)
data.head()

### 4.1.2 | Supplementary Location Information (Mixed Dataset)
As our dataset was limited in useful features, we wanted to add supplementary information. Using a geopandas package, we were able to get more information about the location of the fire using latitude and longitude. We were able to add information like the country, continent and estimated population of in the area, which could be useful to determine the intensity of the fire.

In [ ]:
# Converting latitude and longitude values to location values

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data['longitude'], data['latitude']), crs={'init': 'epsg:4326'})
result = gpd.sjoin(gdf, world, how='left')
result.head()

In [ ]:
result.drop(['geometry','index_right','iso_a3','gdp_md_est'],axis=1,inplace=True)
result.head()

In [ ]:
# Check for null values
result.isnull().sum()

In [ ]:
# Above means there are null values present. 
# Solution: remove rows
result.dropna(inplace=True)
result.reset_index(drop=True) # reset index of dataframe to account for missing values
result.isnull().sum()

In [ ]:
# Visualize clean dataset
result.head()

In [ ]:
# One Hot Encoding
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')

ohe_confidence = pd.DataFrame(enc.fit_transform(result[['confidence']]).toarray())
ohe_confidence.columns = enc.get_feature_names(['confidence'])
#ohe_confidence.head()

ohe_daynight = pd.DataFrame(enc.fit_transform(result[['daynight']]).toarray())
ohe_daynight.columns = enc.get_feature_names(['daynight'])
# ohe_daynight.head()

ohe_continent = pd.DataFrame(enc.fit_transform(result[['continent']]).toarray())
ohe_continent.columns = enc.get_feature_names(['continent'])
# ohe_continent.head()

ohe_name = pd.DataFrame(enc.fit_transform(result[['name']]).toarray())
ohe_name.columns = enc.get_feature_names(['name'])
# ohe_name.head()

In [ ]:
# Concatenate OHE variables with dataframe
new_data = pd.concat([result,ohe_confidence,ohe_daynight,ohe_continent,ohe_name], axis=1)
new_data.drop(['confidence', 'daynight','continent','name'], axis=1, inplace=True)
new_data.dropna(inplace=True)
new_data.head()

In [ ]:
# Check datatypes of new dataframe
new_data.dtypes

In [ ]:
# Standardization can only be done on numeric data - hence, columns not of int or float type should be removed.
new_data.drop(['period','acq_date','latitude','longitude','version'],axis=1,inplace=True)
new_data.dtypes

In [ ]:
#create a dataset to reference to
data_mixed = new_data
print(data_mixed.head())

### 4.1.3 | Converting Continuous Features into Categorical Features (Binned Dataset)
To make the model more accurate, we attempted to convert continuous features into categorical features by splitting the datapoints into 5 bins of equally spaced out ranges of continuous data. We will then use OHE to convert these bins into dummy variables. This is so that we can simplify and standardize the type of features which is fed to the models by converting all the features (X) to binary variables. We will be training our models using this preprocessed fully categorical dataset, as well as the mixed continous-categorical dataset.

In [ ]:
# Generate labels for modified dataset
mod_var = [[],[],[],[],[],[]]
var = ['bright_ti4','scan','track','bright_ti5','pop_est']
quantile = ['_20p','_40p','_60p','_80p','_100p']
quartile = ['_25p','_50p','_75p','_100p']

for x in range(len(var)):
    mod_var[x] = [var[x] + value for value in quantile]
    print (mod_var[x])
mod_var[-1]=['acq_time'+value for value in quartile]
print (mod_var[-1])

In [ ]:
# Convert input to categorical input

data_bright_ti4,data_bright_ti4_intervals=pd.qcut(new_data['bright_ti4'], 5, retbins=True,
                        labels=mod_var[0])
data_scan,data_scan_intervals=pd.qcut(new_data['scan'], 5, retbins=True,
                        labels=mod_var[1])
data_track,data_track_intervals=pd.qcut(new_data['track'], 5, retbins=True,
                        labels=mod_var[2])
data_bright_ti5,data_bright_ti5_intervals=pd.qcut(new_data['bright_ti5'], 5, retbins=True,
                        labels=mod_var[3])
data_pop_est,data_pop_est_intervals=pd.qcut(new_data['pop_est'], 5, retbins=True,
                        labels=mod_var[4])
data_acq_time,data_acq_time_intervals=pd.qcut(new_data['acq_time'], 4, retbins=True,
                        labels=mod_var[5])

In [ ]:
#Concatenate all to a single DF
cont_var = pd.concat([data_bright_ti4,data_scan,data_track,data_bright_ti5,data_pop_est,data_acq_time], axis=1)

In [ ]:
cont_var.dropna(inplace= True)

In [ ]:
# One-Hot Encoding
ohe_bright_ti4 = pd.DataFrame(enc.fit_transform(cont_var[['bright_ti4']]).toarray())
ohe_bright_ti4.columns = enc.get_feature_names(['bright_ti4'])

ohe_scan = pd.DataFrame(enc.fit_transform(cont_var[['scan']]).toarray())
ohe_scan.columns = enc.get_feature_names(['scan'])

ohe_track = pd.DataFrame(enc.fit_transform(cont_var[['track']]).toarray())
ohe_track.columns = enc.get_feature_names(['track'])

ohe_bright_ti5 = pd.DataFrame(enc.fit_transform(cont_var[['bright_ti5']]).toarray())
ohe_bright_ti5.columns = enc.get_feature_names(['bright_ti5'])

ohe_pop_est = pd.DataFrame(enc.fit_transform(cont_var[['pop_est']]).toarray())
ohe_pop_est.columns = enc.get_feature_names(['pop_est'])

ohe_acq_time = pd.DataFrame(enc.fit_transform(cont_var[['acq_time']]).toarray())
ohe_acq_time.columns = enc.get_feature_names(['acq_time'])

In [ ]:
new_data.drop(['bright_ti4','scan','track','acq_time','bright_ti5','pop_est'],axis=1,inplace=True)
new_data = pd.concat([ohe_bright_ti4,ohe_scan,ohe_track,ohe_bright_ti5,ohe_pop_est,ohe_acq_time,new_data],axis=1)

In [ ]:
new_data.head()

In [ ]:
new_data.dropna(inplace=True)

In [ ]:
#create a dataset to reference to
data_binned = new_data

## 4.2 | Feature Selection

With our two datasets, we shall do some feature selection to make our models run faster and smoother. 

In [ ]:
# Binned Dataset
Xmix = data_mixed.drop(['frp'], axis=1)
ymix = pd.DataFrame(data_mixed['frp'])

# Binned Dataset
Xbin = data_binned.drop(['frp'], axis=1)
ybin = pd.DataFrame(data_binned['frp'])

### 4.2.1 | Covariance Heatmap

In [ ]:
##z-scoring standardization

from sklearn.preprocessing import StandardScaler, Normalizer
def standardize(df):
  # create a scaler object
  std_scaler = StandardScaler()
  # fit and transform the data
  return pd.DataFrame(std_scaler.fit_transform(df), columns=df.columns)

print(data_mixed.head())
X_cleaned  = standardize(data_mixed)
print(X_cleaned[:5])

In [ ]:
Xdata = X_cleaned.copy()
plt.figure(1)
fig, ax = plt.subplots(figsize=(25,25)) 
sns.set()
ax = sns.heatmap(Xdata.corr(method='pearson'),vmin=0,cmap="YlGnBu",annot=True,ax=ax)

### 4.2.2 | Random Forest Model

Another model we utilised is the Random Forest model, which comprises of a user-determined number of decision trees. 

It works through the following steps:
1. A random sample is selected from the given dataset.
2. From this random sample, a decision tree is crafted. Assuming there are n random samples selected, there will be n decision trees. (The argument n_estimators allows the user to choose the value of n.)
3. When the Random Forest model makes a prediction, each decision tree will then produce a predicted result. Voting will commence for all n decision trees.
4. Once all n predicted results have been voted upon, the highest voted result is the final prediction.

The model has proven to be both accurate and robust due to the number of decision trees involved; additionally, due to it taking the average of all predictions by the decision trees, it is not affected by overfitting.

Sources: 
- [Random Forest in Python](https://towardsdatascience.com/random-forest-in-python-24d0893d51c0)
- [Understanding Random Forests Classifiers in Python](https://www.datacamp.com/community/tutorials/random-forests-classifier-python)

In [ ]:
##convert to numpy to use with randomforest

X = Xmix
y = ymix

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X.values, y.values.ravel());

# get importance
importance = rf.feature_importances_
# print(importance)
# summarize feature importance
important_features_dict = {}
for i,v in enumerate(importance):
    important_features_dict[X.columns[i]] = v

sorted_values = sorted(important_features_dict.values(),reverse=True) # Sort the values
sorted_dict = {}

for i in sorted_values:
    for k in important_features_dict.keys():
        if important_features_dict[k] == i:
            sorted_dict[k] = important_features_dict[k]
            break
         
toplist = []
for i in sorted_dict:
    #print(i,sorted_dict[i])
    toplist.append(i)

first3vals = toplist[:3]
print("The top 3 factors correlating to the Fire Radiative Power are %s, %s, %s" % ( first3vals[0], first3vals[1] , first3vals[2] ))

# 5 | Data Analysis - Supervised Learning Models
In this section, we shall be using our dataset to train three different models to attempt to predict a value of FRP (Fire Radiative Power). Fire Radiative Power is the detected thermal energy of the fire, which is indicative of the size and intensity of the wildfire.

<p> From the feature selection, the relevant features that we are using to predict FRP values would be bright_ti4 and bright_ti5, and a high confidence of ths satelite in the conditions for measurement.

In [ ]:
data_xs_mixed = 
data_y_mixed = 

data_xs_binned = 
data_y_binned =

print(data_xs[:5], data_xs.shape)
print(data_y[:5], data_y.shape)

In [ ]:
data_xs.dtypes

In [ ]:
#Visualization of our cleaned dataset
def scatterplot_XY(x, y, x_label):
  plt.scatter(x, y, s=60, c='r', marker='+', label='Class0')
  #plt.xlabel(x.keys())
  plt.ylabel('frp') 
  plt.xlabel(x_label)
  plt.show()
  plt.clf()

for column in data_xs:
  x= data_xs[column]
  y = data_y
  scatterplot_XY(x, y, column)

In [ ]:
#TO DO, or TO DELETE: categorical data visualization. 
#sns.relplot(kind ='bubble', x = data_xs['confidence_high'], y = data_y)
##sns.scatterplot(data=data, x="gdpPercap", y="lifeExp", size="pop", legend=False, sizes=data_xs)

dfviolin = pd.DataFrame([data_xs['confidence_high'].values, data_y.values]).transpose()
#dfviolin = pd.DataFrame(data=np.concatenate((data_xs['confidence_high'],y_pred),axis=1), columns=["Actual","Predicted"])

print(dfviolin[:5])

sns.violinplot(data = dfviolin, x='0', y = '1')

In [ ]:
# due to the high density of our points, we shall reduce their opacity and visualize them in terms of density
#visualization
def scatterplot_XY(x, y,x_label = "x_variable"):
  plt.scatter(x, y, s=60, c='r', marker='+', alpha = 0.1, label='Class0')
  #plt.xlabel(x.keys())
  plt.ylabel('frp') 
  plt.xlabel(x_label)
  plt.show()
  plt.clf()


for column in data_xs:
  x= data_xs[column]
  y = data_y
  scatterplot_XY(x, y, column)

## 5.1 | Approach
This is a regression problem, where we are attempting to predict a continous variable y = FRP, from 3 different features X = ['bright_ti4', 'bright_ti5', 'confidence_high']

We will be applying 3 different regression models to attempt to quantify a relationship between the X and y variables. The models are: 
1.   Linear Regression
2.   Polynomial Regression
3.   Random Forest




### 5.1.1 | Train-Validation-Test Split

Before we work on the dataset, we are going to split the dataset into train sets and test sets. The train set will be used for creating and fitting our model parameters, while the test data is used to evaluate the accuracy and effectiveness of our model for predicting FRP values. 

For models that have hyperparameters to be calibrated, we shall also split the train set from above into train-validation sets and conduct cross-fold validation to calibrate the hyperparameters.

In [ ]:
# data for the models here
data_xs = data_xs
data_y = data_y

#train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_xs, data_y, test_size=0.2, random_state=0)

print("Shape of Train Dataset (X,y):", X_train.shape, y_train.shape)
print("Shape of Test Dataset (X,y):", X_test.shape, y_test.shape)

In [ ]:
X_train.dtypes

In [ ]:
#train-validation Cross Validation with 4 folds
def crossvald(data_xs, data_y):
  ### n_folds = 4
  X_b1, X_b2, y_b1, y_b2 = train_test_split(data_xs, data_y, test_size=0.5, random_state=0)
  X_1, X_2, y_1, y_2 = train_test_split(X_b1, y_b1, test_size=0.5, random_state=0)
  X_3, X_4, y_3, y_4 = train_test_split(X_b2, y_b2, test_size=0.5, random_state=0)

  cross_sets ={1:{'train_X': np.concatenate((X_2,X_3,X_4)),
                  'train_y': np.concatenate((y_2,y_3,y_4)),
                  'val_X': X_1,
                  'val_y': y_1,
                    },
               2:{'train_X': np.concatenate((X_1,X_3,X_4)),
                  'train_y': np.concatenate((y_1,y_3,y_4)),
                  'val_X': X_2,
                  'val_y': y_2,
                    },
               3:{'train_X': np.concatenate((X_2,X_1,X_4)),
                  'train_y': np.concatenate((y_2,y_1,y_4)),
                  'val_X': X_3,
                  'val_y': y_3,
                    },
               4:{'train_X': np.concatenate((X_2,X_3,X_1)),
                  'train_y': np.concatenate((y_2,y_3,y_1)),
                  'val_X': X_4,
                  'val_y': y_4,
                    },
               }
  return cross_sets

CValSets = crossvald(X_train, y_train)

In [ ]:
print("Shape of the Train Set 1, (X,y):", CValSets[1]['train_X'].shape, CValSets[1]['train_y'].shape)
print("Shape of the Validation Set 1, (X,y):",CValSets[1]['val_X'].shape, CValSets[1]['val_y'].shape)

### 5.1.2 | Accuracy Score Formulas

To evaluate the accuracy of the regression model, we shall be using the following accuracy and loss formula across all the models.

Percent Error: 
$$
P.E. (y_{pred}, y_{actual}) 
= \frac{error}{actual} 
= \frac{|y_{pred}- y_{actual}| }{ | y_{pred}+y_{actual}|/2} 
$$

While the actual value is  $y_{actual}$,  the mean of $y_{pred}$ and $y_{actual}$ is used to prevent division by zero errors. 

\
Accuracy:
<br>
\begin{align}
 Acc. (y_{pred}, y_{actual}) 
& = 1- mean (P.E.) \\
& = 1- \frac{1}{n} *  \sum\frac{|y_{pred}- y_{actual}| }{ | y_{pred}+y_{actual}|/2} 
\end{align}

Loss:
$$
L(y_{pred}, y_{actual}) = \frac{1}{n} *  \sum (y_{pred} - y_{actual})^2
$$

We shall be using the sci-kit learn library for implementation of all the models.


In [ ]:
def loss_mse(y_actual, y_pred):
  n = len(y_pred)
  total = (y_pred-y_actual).T.dot(y_pred-y_actual)
  loss = 1/n* total
  return loss

def mape (y_actual, y_pred):
  # Calculate mean absolute percentage error (MAPE) modified to account for zeros in the actual readings
  mape = 100 * (abs(y_pred-y_actual) / ((abs(y_actual+y_pred))/2)).values
  return mape

def accuracy(y_actual, y_pred):
  mape = 100 * (abs(y_pred-y_actual) / ((abs(y_actual+y_pred))/2)).values
  # Calculate and display accuracy
  accuracy = 100 - np.mean(mape)
  return accuracy 

In [ ]:
## Print Formats 
loss = loss_mse(y_test, y_pred)
print("Loss Test Set:", loss)

y_pred_train = reg_train.predict(X_train)
loss = loss_mse(y_train, y_pred_train)
print("Loss Train set:", loss)


print('Accuracy:', round(accuracy(y_test, y_pred), 2), '%.')

In [ ]:
from sklearn import metrics
def sklearn_metrics(y_actual, y_pred):

  print('Mean Absolute Error:', metrics.mean_absolute_error(y_actual, y_pred))
  print('Mean Squared Error (Loss that we have defined):', metrics.mean_squared_error(y_actual, y_pred))
  print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_actual, y_pred)))
  print('Accuracy (sklearn):', metrics.accuracy_score(y_actual, y_pred) )

## 5.1 | Linear Regression Model
Firstly, we shall be attempting to use a simple linear regression model to predict the continuous FRP values

In [ ]:
from sklearn.linear_model import LinearRegression
 
def linear_reg(X,y):
  reg = LinearRegression().fit(X, y)

  #R^2 value, which is between -1 and 1
  print("Model Score (R^2):", reg.score(X, y))

  print("Coeffcients", list(X.columns),": ", reg.coef_)
  print("Intercept:", reg.intercept_)
  return reg

reg_train = linear_reg(X_train, y_train)

In [ ]:
y_pred = reg_train.predict(X_test)

print(y_pred[:5], type(y_pred), y_pred.shape)

df = pd.DataFrame(data=np.concatenate((y_test,y_pred),axis=1), columns=["Actual","Predicted"])

print (df[:5])

loss = loss_mse(y_pred, y_test)
print("Loss Test Set", loss)

y_pred_train = reg_train.predict(X_train)
loss = loss_mse(y_pred_train, y_train)
print("Loss Train set:", loss)

from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


print(accuracy(y_test, y_pred))

In [ ]:
#attempt at visualization
def scatterplot_wmodel(x,y,f_pred):
  plt.scatter(x, y, s=60, c='r', marker='+', label='Class0', alpha = 0.001)
  #plt.xlabel(x.keys())
  plt.ylabel('frp') 

  x_pred = np.linspace(min(x),max(x),100)
  y_pred = f_pred(x_pred)
  plt.plot(x_pred, y_pred)
  plt.show()
  plt.clf()

i = 0
for column in data_xs:
  x= data_xs[column]
  y = data_y  
  x_pred = np.linspace(min(x),max(x),100)
  y_pred = reg_train.coef_[0][i]*x_pred + reg_train.intercept_
  plt.plot(x_pred,y_pred) 
  
  plt.scatter(x, y, s=60, c='r', marker='+', label='Class0')
  plt.show() 
  i += 1
  plt.clf()



## 5.3 | Random Forest/Decision Tree
Attempt to use random forest/Decision tree to model the variables

implemented using this: https://towardsdatascience.com/random-forest-in-python-24d0893d51c0

In [ ]:
print(type(y_train))
print(type(X_train))

##convert to numpy to use with randomforest
X_trainn = X_train.values
y_trainn = y_train.values.reshape(-1,)

print(type(y_trainn), y_trainn.shape)
print(type(X_trainn), X_trainn.shape)


In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 20, random_state = 42)
# Train the model on training data
rf.fit(X_trainn, y_trainn);

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test.values)
# Calculate the absolute errors
errors = abs(predictions - y_test.values.reshape(-1,))
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))


df = pd.DataFrame(data=np.concatenate((y_test,predictions.reshape(-1,1), errors.reshape(-1,1)),axis=1), columns=["Actual","Predicted", "Absolute Error"])

print (df[:5])

In [ ]:
# Calculate mean absolute percentage error (MAPE) modified to account for zeros in the actual readings
mape = 100 * (errors / ((abs(y_test.values.reshape(-1,))+abs(predictions))/2))
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)

print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
feature_list = X_train.columns

# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)


In [ ]:
#run with caution Already generated in folder
# Use dot file to create a graph
# oh no this takes forever to excute - estimated time = 4mins
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file - estimated time >15mins
graph.write_png('tree.png')

##Neural Network Implementation
Try a neural network I guess here

In [ ]:
#turn frp into categorical data? 
#I honestly have no clue how to do it
#cos neural network is mostly for y is categorical data I feel


## Polynomial Model with Cross-Fold Validation
Cross-Fold Validation is used to optimize for the d variable

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
d = 4

def poly_reg(X_train,y_train,d, X_test, y_test):
  poly_reg = PolynomialFeatures(degree=d)
  X_poly = poly_reg.fit_transform(X_train)

  reg = LinearRegression().fit(X_poly, y_train)

  X_poly_t = poly_reg.fit_transform(X_test)
  #R^2 value, which is between -1 and 1
  print("Training Loss of R^2:", reg.score(X_poly, y_train))
  print("Test Loss of R^2:", reg.score(X_poly_t, y_test))
  #print("Coeffcients [x1,x2] : ", reg.coef_)
  #print("Intercept:", reg.intercept_)
  y_pred_t = reg.predict(X_poly_t)

  #loss
  mse = metrics.mean_squared_error(y_test, y_pred_t)
  #score, this was the original metric i used 
  reg.score(X_poly_t, y_test)

  return reg, X_poly, mse

reg_train, X_poly, test_score = poly_reg(X_train, y_train, 4, X_test, y_test)

#R^2 value, which is between -1 and 1
#print("Model Loss of R^2:", poly_reg.score(X, y))


In [ ]:
splitdata = crossvald(X_train, y_train)

len(splitdata)

In [ ]:
##selection of ideal d
def bestd (dim, splitdata):
  meanscore = 0
  bestscore = 0
  best_set = None
  best_model = None
  for setno, setdata in splitdata.items():
    #print(setno)
    reg_train, X_poly, test_score = poly_reg(setdata['train_X'], setdata['train_y'], dim, setdata['val_X'], setdata['val_y'])
    meanscore += test_score/len(splitdata)
    if (test_score < bestscore):
      bestscore = test_score
      best_set = setno
      best_model = reg_train

  return meanscore, bestscore, best_model, best_set

yd =[]
for d in range(1,20):

  meanscore, bestscore, best_model, best_set = bestd (d, splitdata)
  yd.append({'meanscore':meanscore, 'bestscore':bestscore, 'best_model': best_model, 'best_set':best_set})



In [ ]:
# this is the plot when we cross validate for the ideal dimension d

ydf = pd.DataFrame(data=yd)
plt.scatter(range(1,20), ydf['meanscore'])
plt.show


best_dim = int(ydf[['meanscore']].idxmin()+1)
best_reg = ydf.at[int(best_dim-1),'best_model']
best_set = ydf.at[int(best_dim-1),'best_model']


print(ydf)
print(min(ydf['meanscore']),'best dimension:', best_dim)

In [ ]:
polyreg = PolynomialFeatures(degree=best_dim)
X_poly_t = polyreg.fit_transform(X_test)
X_poly = polyreg.fit_transform(X_train)
print(X_poly_t.shape)



y_pred = best_reg.predict(X_poly_t)
print("Coeffcients [x1,x2] : ", best_reg.coef_.shape)
print("Intercept:", best_reg.intercept_)


print(y_pred[:5], type(y_pred), y_pred.shape)

df = pd.DataFrame(data=np.concatenate((y_test,y_pred, abs(y_pred-y_test)),axis=1), columns=["Actual","Predicted", "Error"])

print (df[:5])

def loss_mse(y_pred, y_test):
  n = len(y_pred)
  total = (y_pred-y_test).T.dot(y_pred-y_test)
  loss = 1/n* total
  return loss

loss = loss_mse(y_pred, y_test)
print("Loss Test Set", loss)

y_pred_train = best_reg.predict(X_poly)
loss = loss_mse(y_pred_train, y_train)
print("Loss Train set:", loss)

from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


# Calculate mean absolute percentage error (MAPE) modified to account for zeros in the actual readings
mape = 100 * (abs(y_pred-y_test) / ((abs(y_test+y_pred))/2)).values
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)

print('Accuracy:', round(accuracy, 2), '%.')

# 6 | Models - Unsupervised Learning
K-means clustering to identify hotspots from one day of datapoints

In [ ]:
data[:5]

In [ ]:
#cleaning the data, the relevant data is just lat, long and frp, confidence
#need to restructure the data such that it separates out the dates

data_c = data[['latitude', 'longitude', 'frp', 'confidence']].groupby(data['acq_date'])

data_w = list(data_c)[0][1]

In [ ]:
fig1 = px.scatter_geo(data_w, 
                    lat='latitude', 
                    lon='longitude', 
                    scope='asia',
                    center={'lat':2.2180,'lon':115.6628}, # centered to SEA
                    color='frp',)
fig1.show()

To conduct clustering, we need to find a suitable algorithm which is fast enough to deal with the number of data points we have.
</p>

In [ ]:
print("Number of data points we have=", data_w.shape[0])

In [ ]:
from scipy.cluster.vq import vq, kmeans, whiten
from numpy import array
import matplotlib.pyplot as plt
features  = array([[ 1.9,2.3],
                   [ 1.5,2.5],
                   [ 0.8,0.6],
                   [ 0.4,1.8],
                   [ 0.1,0.1],
                   [ 0.2,1.8],
                   [ 2.0,0.5],
                   [ 0.3,1.5],
                   [ 1.0,1.0]])
whitened = whiten(features)
book = np.array((whitened[0],whitened[2]))
kmeans(whitened,book)

from numpy import random
random.seed((1000,2000))
codes = 3
kmeans(whitened,codes)

# Create 50 datapoints in two clusters a and b
pts = 50
a = np.random.multivariate_normal([0, 0], [[4, 1], [1, 4]], size=pts)
b = np.random.multivariate_normal([30, 10],
                                  [[10, 2], [2, 1]],
                                  size=pts)
features = np.concatenate((a, b))
# Whiten data
whitened = whiten(features)
# Find 2 clusters in the data
codebook, distortion = kmeans(whitened, 2)
# Plot whitened data and cluster centers in red
plt.scatter(whitened[:, 0], whitened[:, 1])
plt.scatter(codebook[:, 0], codebook[:, 1], c='r')
plt.show()

# 7 | I'm gonna try a different dataset because this dataset is so limited :(


# Trash



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(data_xs, data_y, test_size=0.4, random_state=0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


##svm doesnt work because its meant for classification
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)